In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import zipfile
from io import BytesIO
from PIL import Image
import numpy as np
from torchvision.utils import save_image

In [2]:
batch_size = 16
latent_space = 100
_channels = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 300



In [3]:
import zipfile
from PIL import Image
from io import BytesIO
from torchvision import transforms
import torch

def image_batch_generator(zip_file, batch_size, transform=None):
    with zipfile.ZipFile(zip_file, 'r') as z:
        image_files = [f for f in z.namelist() if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        total_images = len(image_files)

        for i in range(0, total_images, batch_size):
            batch_files = image_files[i:i + batch_size]
            images = []
            for filename in batch_files:
                with z.open(filename) as file:
                    img = Image.open(BytesIO(file.read())).convert('RGB')
                    if transform:
                        img = transform(img)
                    images.append(img)
            yield torch.stack(images)



In [4]:
import os
from PIL import Image
import torch

def image_batch_generator(root_folder, batch_size, transform=None):
    all_image_files = []

    # Walk through all subdirectories and collect image files
    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                all_image_files.append(os.path.join(root, file))

    total_images = len(all_image_files)
    
    for i in range(0, total_images, batch_size):
        batch_files = all_image_files[i:i + batch_size]
        images = []
        for filename in batch_files:
            img = Image.open(filename).convert('RGB')
            if transform:
                img = transform(img)
            images.append(torch.tensor(img))
        yield torch.stack(images)


In [5]:
zip_file_path = '/kaggle/input/art-portraits'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

_dataloader = image_batch_generator(zip_file_path, batch_size, transform=transform)

## DESCRIMINATOR

In [6]:
class Descriminator(nn.Module):
    def __init__(self, input_channels):
        super(Descriminator, self).__init__()

        self.channels = input_channels

        self.conv1 = nn.Conv2d(self.channels, 32, kernel_size=3, stride=2, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.activation1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.5)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.activation2 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.3)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.activation3 = nn.LeakyReLU()
        self.dropout3 = nn.Dropout(0.3)

        self.flatten = nn.Flatten(1,-1)

        self.linear1 = nn.Linear(128 * 32 * 32, 2048)
        self.batchnorm4 = nn.BatchNorm1d(2048)
        self.activation4 = nn.LeakyReLU()
        
        self.linear2 = nn.Linear(2048,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation1(x)
        x = self.dropout1(x)
        
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)
        x = self.dropout2(x)

        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.activation3(x)
        x = self.dropout3(x)
        
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.batchnorm4(x)
        x = self.activation4(x)
        
        x = self.linear2(x)
        x = self.sigmoid(x)

        return x

    

In [7]:
# decrimnator = Descriminator(input_channels=3)
# x = decrimnator(_dataloader.__next__())

In [8]:
# x.shape

# GENERATOR

In [9]:
import torch.nn as nn

# output_shape = ( input - 1 ) * stride + output_padding - 2 * padding + kernel_size

class Generator(nn.Module):
    def __init__(self, input_size, output_channels):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.channels = output_channels

        # self.conv1 = nn.ConvTranspose2d(input_size, 1024, 5, 2, 0, bias=False)
        # self.batchnorm1 = nn.BatchNorm2d(1024)
        # self.activation1 = nn.ReLU()

        self.conv2 = nn.ConvTranspose2d(input_size, 512, 4, 2, 0, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(512)
        self.activation2 = nn.ReLU(True)

        self.conv3 = nn.ConvTranspose2d(512, 256, 3, 2, 1,output_padding = 1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(256)
        self.activation3 = nn.ReLU(True)

        self.conv4 = nn.ConvTranspose2d(256, 256, 3, 2, 1,output_padding = 1, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.activation4 = nn.ReLU(True)

        self.conv5 = nn.ConvTranspose2d(256, 128, 3, 2, 1, output_padding = 1,bias=False)
        self.batchnorm5 = nn.BatchNorm2d(128)
        self.activation5 = nn.ReLU(True)

        self.conv6 = nn.ConvTranspose2d(128, 64, 3, 2, 1, output_padding = 1,bias=False)
        self.batchnorm6 = nn.BatchNorm2d(64)
        self.activation6 = nn.ReLU(True)

        self.conv7 = nn.ConvTranspose2d(64, 32, 3, 2, 1,output_padding = 1, bias=False)
        self.batchnorm7 = nn.BatchNorm2d(32)
        self.activation7 = nn.ReLU(True)

        self.conv8 = nn.ConvTranspose2d(32, self.channels, 3, 2, 1, output_padding = 1,bias=False)
        self.batchnorm8 = nn.BatchNorm2d(self.channels)
        self.activation8 = nn.Tanh()

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, self.input_size, 1, 1)
        # x = self.conv1(x)
        # x = self.batchnorm1(x)
        # x = self.activation1(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)

        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.activation3(x)

        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = self.activation4(x)

        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = self.activation5(x)

        x = self.conv6(x)
        x = self.batchnorm6(x)
        x = self.activation6(x)

        x = self.conv7(x)
        x = self.batchnorm7(x)
        x = self.activation7(x)

        x = self.conv8(x)
        x = self.batchnorm8(x)
        x = self.activation8(x)
       
        return x
    

In [10]:
# generator_input = torch.randn(8, 100)  # Batch size of 64, input size of 100
# generator = Generator(input_size=100,output_channels=3)
# output = generator(generator_input)
# print(output.shape)

# GENERATOR USING  UPSAMPLING

In [11]:
import torch.nn as nn

class UpGenerator(nn.Module):
    def __init__(self, input_size=100, output_channels=3):
        super(UpGenerator, self).__init__()
        self.input_size = input_size
        self.channels = output_channels

        self.fc = nn.Linear(input_size, 512 * 8 * 8)

        self.conv1 = nn.Conv2d(512, 256, 3, 1, 1, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(256)
        self.activation1 = nn.ReLU()


        self.upsampling1 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv2 = nn.Conv2d(256, 128, 3, 1, 1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(128)
        self.activation2 = nn.ReLU()

        self.upsampling2 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv3 = nn.Conv2d(128, 64, 3, 1, 1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.activation3 = nn.ReLU()

        self.upsampling3 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv4 = nn.Conv2d(64, 32, 3, 1, 1, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(32)
        self.activation4 = nn.ReLU()

        self.upsampling4 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv5 = nn.Conv2d(32, 16, 3, 1, 1, bias=False)
        self.batchnorm5 = nn.BatchNorm2d(16)
        self.activation5 = nn.ReLU()

        self.upsampling5 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv6 = nn.Conv2d(16, self.channels, 3, 1, 1, bias=False)
        self.activation6 = nn.Tanh()


    def forward(self, x):
        batch_size = x.size(0)
        x = self.fc(x).view(batch_size, 512, 8, 8)

        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation1(x)

        x = self.upsampling1(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)

        x = self.upsampling2(x)
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.activation3(x)

        x = self.upsampling3(x)
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = self.activation4(x)

        x = self.upsampling4(x)
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = self.activation5(x)

        x = self.upsampling5(x)
        x = self.conv6(x)
        x = self.activation6(x)

        return x

# generator_input = torch.randn(8, 100)  # Batch size of 64, input size of 100
# generator = UpGenerator(input_size=100,output_channels=3)
# output = generator(generator_input)
# print(output.shape)

In [12]:

class DCGAN(nn.Module):
    def __init__(self, discriminator,generator, latent_dim, device, lr_gen=0.0002, lr_disc=0.0002, betas=(0.5, 0.999)):
        super(DCGAN, self).__init__()
        self.generator = generator.to(device)
        self.discriminator = discriminator.to(device)
        self.latent_dim = latent_dim
        self.device = device
        
        self.optim_gen = optim.Adam(self.generator.parameters(), lr=lr_gen, betas=betas)
        self.optim_disc = optim.Adam(self.discriminator.parameters(), lr=lr_disc, betas=betas)
        
        self.criterion = nn.BCELoss()
        self.loss_fn = nn.BCELoss()
        
    def train_step(self, real_images):
        batch_size = real_images.shape[0]
        real_images = real_images.to(self.device)

        random_latent_vectors = torch.randn(batch_size, self.latent_dim, device=self.device)

        generated_images = self.generator(random_latent_vectors)

        real_predictions = self.discriminator(real_images)
        fake_predictions = self.discriminator(generated_images.detach())  # Detach to avoid backprop through generator

        # Labels and label noise
        real_labels = torch.ones_like(real_predictions, device=self.device)
        real_noisy_labels = real_labels + 0.1 * torch.rand_like(real_labels, device=self.device)
        fake_labels = torch.zeros_like(fake_predictions, device=self.device)
        fake_noisy_labels = fake_labels - 0.1 * torch.rand_like(fake_labels, device=self.device)


        real_noisy_labels = torch.clamp(real_noisy_labels,0,1)
        fake_noisy_labels = torch.clamp(fake_noisy_labels,0,1)
        
        d_real_loss = self.loss_fn(real_predictions, real_noisy_labels)
        d_fake_loss = self.loss_fn(fake_predictions, fake_noisy_labels)
        d_loss = (d_real_loss + d_fake_loss) / 2.0


        self.optim_disc.zero_grad()
        d_loss.backward()
        self.optim_disc.step()

        fake_predictions = self.discriminator(generated_images)
        g_loss = self.criterion(fake_predictions, real_labels) #+ nn.SmoothL1Loss()(generated_images, real_images)

        # Backpropagation for generator
        self.optim_gen.zero_grad()
        g_loss.backward()
        self.optim_gen.step()

        return {"d_loss": d_loss.item(), "g_loss": g_loss.item()}
    

    def generate_images(self, num_images=1, save_path=None,epoch=None):
        self.generator.eval()
        if epoch is None:
            epoch = "_"	
            
        if os.path.exists(save_path) == False:
            os.makedirs(save_path)

        with torch.no_grad():
            noise = torch.randn(num_images, self.latent_dim).to(self.device)
            generated_images = self.generator(noise)
            
            if save_path is not None:
                for i, image in enumerate(generated_images):
                    if os.path.exists(save_path) == False:
                        os.makedirs(save_path)
                    save_image(image, f"{save_path}/EPOCH_{epoch}_image_{i}.png")
            
            return generated_images

In [13]:
# def weights_init_normal(m):
#     if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
#         nn.init.normal_(m.weight, mean=0.0, std=0.02)
#         if m.bias is not None:
#             nn.init.zeros_(m.bias)
# latent_space = 250

# descriminator = Descriminator(input_channels=_channels).apply(weights_init_normal)
# generator = Generator(input_size=latent_space,output_channels=_channels).apply(weights_init_normal)

# DCGAN = DCGAN(descriminator, generator, latent_space, device, lr_gen= 0.00002, lr_disc=0.00002, betas=(0.5, 0.999))

def weights_init_glorot(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


descriminator = Descriminator(input_channels=_channels).apply(weights_init_glorot)
generator = Generator(input_size=latent_space,output_channels=_channels).apply(weights_init_glorot)

DCGAN = DCGAN(descriminator, generator, latent_space, device, lr_gen=2e-3, lr_disc=2e-3, betas=(0.5, 0.999))

In [14]:
_dataloader = image_batch_generator(zip_file_path, 16, transform=transform)

In [15]:
DCGAN.train_step(_dataloader.__next__())


/tmp/ipykernel_472/3432998535.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images.append(torch.tensor(img))


{'d_loss': 0.8332287073135376, 'g_loss': 4.658342361450195}

In [ ]:
# import shutil
# import os

# # Specify the path to the folder you want to delete
# folder_path = '/kaggle/working/generated_images'

# # Check if the folder exists
# if os.path.exists(folder_path):
#     # Remove the folder and all its contents
#     shutil.rmtree(folder_path)

In [16]:
for i in range(num_epochs):
    x=0
    _dataloader = image_batch_generator(zip_file_path, 32, transform=transform)
    for idx,j in enumerate(_dataloader):
        x = DCGAN.train_step(j)
        print(f"epoch :{i} batch: {idx}",end ="\r")
    print(f"epoch {i}: {x}")
    if i%15 == 0:
        torch.save(DCGAN.state_dict(), 'model_state_dict.pth')
    DCGAN.generate_images(num_images=1, save_path="generated_images",epoch=i)

/tmp/ipykernel_472/3432998535.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images.append(torch.tensor(img))


epoch 0: {'d_loss': 1.3815033435821533, 'g_loss': 6.308757305145264}
epoch 1: {'d_loss': 1.3254845142364502, 'g_loss': 3.7382590770721436}
epoch 2: {'d_loss': 1.0476481914520264, 'g_loss': 4.312037944793701}
epoch 3: {'d_loss': 0.5357766151428223, 'g_loss': 4.355941295623779}


KeyboardInterrupt: 

In [ ]:
# Save only the model's state dict
# torch.save(DCGAN.state_dict(), 'model_state_dict.pth')
# os.remove("/kaggle/working/model_state_dict.pth")
# torch.save(DCGAN.state_dict(), 'model_state_dict.pth')

In [ ]:
DCGAN.generate_images(num_images=1, save_path="generated_images",epoch=4001)

In [ ]:
sum(i.numel() for i in DCGAN.parameters())